**If u like this notebook pls upvote!!!!**

In [ ]:
#Install required modules

!pip install segmentation-models-pytorch
!pip install albumentations
!pip install opencv-python-headless

In [ ]:
#Some of these matter, some of these dont. Most of them probably dont cuz I copied these from someone else. 
#You can remove these if u want idk why u would tho
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, Input, Concatenate
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import MeanIoU

In [ ]:
#Create the base data frame for the data. IF you have multiple files you will need to change base directory.
import os
import pandas as pd
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
def Create_Directory_DataFrame(typ='train', dataset_type='idrid'):
    """
    Create DataFrame for either IDRiD or DDR dataset
    """
    rows = []
    
    if dataset_type == 'idrid':
        # IDRiD dataset structure
        basedir = '/kaggle/input/indian-diabetic-retinopathy-image-dataset/A.%20Segmentation/A. Segmentation/1. Original Images/'
        
        for Class in os.listdir(basedir):
            for location in os.listdir(os.path.join(basedir, Class)):
                if location.endswith('.jpg'):
                    class_split = Class.split(' ')
                    class_name = class_split[1] if len(class_split) > 1 else class_split[0]
                    
                    rows.append({
                        'Dataset': 'idrid',
                        'Class': class_name,
                        'Location': os.path.join(basedir, Class, location),
                        'ID': location.split('.')[0].split('_')[1] if '_' in location else location.split('.')[0],
                        'Type': 'train'  # IDRiD doesn't have separate validation, we'll split it
                    })
    
    elif dataset_type == 'ddr':
        # DDR dataset structure - fixed the f-string issue
        basedir = f"/kaggle/input/ddr-segmentation/lesion_segmentation/{typ}/"
        image_dir = os.path.join(basedir, 'image')
        
        if not os.path.exists(image_dir):
            print(f"Warning: DDR {typ} image directory not found: {image_dir}")
            return pd.DataFrame(rows)
        
        # Get all JPEG images
        for img_path in glob(os.path.join(image_dir, '*.jpg')):
            img_name = os.path.basename(img_path)
            img_id = img_name.split('.')[0]  # Remove .jpeg extension
            
            rows.append({
                'Dataset': 'ddr',
                'Class': 'DR',  # DDR doesn't have separate classes for images
                'Location': img_path,
                'ID': img_id,
                'Type': typ
            })
    
    df = pd.DataFrame(rows)
    if len(df) > 0:
        df = df.sample(frac=1).reset_index(drop=True)
    return df

def Create_Directory_Array(typ='train', dataset_type='idrid'):
    """
    Create mask DataFrame for either IDRiD or DDR dataset
    """
    rows = []
    
    if dataset_type == 'idrid':
        # IDRiD mask structure
        basedir = '/kaggle/input/indian-diabetic-retinopathy-image-dataset/A.%20Segmentation/A. Segmentation/2. All Segmentation Groundtruths/'
        
        for Class in os.listdir(basedir):
            class_path = os.path.join(basedir, Class)
            
            if os.path.isdir(class_path):
                class_parts = Class.split(' ')
                class_label = class_parts[1] if len(class_parts) > 1 else class_parts[0]
                
                for subclass in os.listdir(class_path):
                    subclass_path = os.path.join(class_path, subclass)
                    
                    if os.path.isdir(subclass_path):
                        for location in os.listdir(subclass_path):
                            if location.endswith('.tif'):
                                rows.append({
                                    'Dataset': 'idrid',
                                    'Class': class_label,
                                    'SubClass': subclass,
                                    'Location': os.path.join(subclass_path, location),
                                    'ID': location.split('.')[0].split('_')[1] if '_' in location else location.split('.')[0],
                                    'Type': 'train'
                                })
    
    elif dataset_type == 'ddr':
        # DDR mask structure - now supports both train and validation
        basedir = f"/kaggle/input/ddr-segmentation/lesion_segmentation/{typ}/label/"
        if typ == 'valid':
            basedir = f"/kaggle/input/ddr-segmentation/lesion_segmentation/{typ}/segmentation label/"
        
        if not os.path.exists(basedir):
            print(f"Warning: DDR {typ} label directory not found: {basedir}")
            return rows
            
        lesion_types = ['MA', 'EX', 'HE', 'SE']  # Microaneurysms, Exudates, Haemorrhages, Soft Exudates
        
        for lesion_type in lesion_types:
            lesion_dir = os.path.join(basedir, lesion_type)
            if not os.path.exists(lesion_dir):
                print(f"Warning: DDR {typ} lesion directory not found: {lesion_dir}")
                continue
                
            for mask_path in glob(os.path.join(lesion_dir, '*.tif')):
                mask_name = os.path.basename(mask_path)
                mask_id = mask_name.split('.')[0]  # Remove .tif extension
                
                # Map DDR lesion types to our standard class names
                class_mapping = {
                    'MA': 'Microaneurysms',
                    'EX': 'Hard Exudates', 
                    'HE': 'Haemorrhages',
                    'SE': 'Soft Exudates'
                }
                
                rows.append({
                    'Dataset': 'ddr',
                    'Class': class_mapping[lesion_type],
                    'SubClass': class_mapping[lesion_type],
                    'Location': mask_path,
                    'ID': mask_id,
                    'Type': typ
                })
    
    return rows

# Let's debug step by step
print("=== Creating DataFrames ===")

# Create DataFrames for both datasets and both types
print("Creating IDRiD original DataFrame...")
df_original_idrid = Create_Directory_DataFrame('train', 'idrid')
print(f"IDRiD original shape: {df_original_idrid.shape}")

print("\nCreating DDR train original DataFrame...")
df_original_ddr_train = Create_Directory_DataFrame('train', 'ddr')
print(f"DDR train original shape: {df_original_ddr_train.shape}")

print("\nCreating DDR validation original DataFrame...")
df_original_ddr_val = Create_Directory_DataFrame('valid', 'ddr')
print(f"DDR validation original shape: {df_original_ddr_val.shape}")

# Create mask DataFrames for both datasets and both types
print("\nCreating IDRiD mask array...")
mask_array_idrid = Create_Directory_Array('train', 'idrid')
df_mask_idrid = pd.DataFrame(mask_array_idrid)
print(f"IDRiD mask shape: {df_mask_idrid.shape}")

print("\nCreating DDR train mask array...")
mask_array_ddr_train = Create_Directory_Array('train', 'ddr')
df_mask_ddr_train = pd.DataFrame(mask_array_ddr_train)
print(f"DDR train mask shape: {df_mask_ddr_train.shape}")

print("\nCreating DDR validation mask array...")
mask_array_ddr_val = Create_Directory_Array('valid', 'ddr')
df_mask_ddr_val = pd.DataFrame(mask_array_ddr_val)
print(f"DDR validation mask shape: {df_mask_ddr_val.shape}")

# Merge function for any dataset and type
def merge_datasets(original_df, mask_df, dataset_name):
    """Helper function to merge original and mask dataframes"""
    if len(original_df) > 0 and len(mask_df) > 0 and 'ID' in original_df.columns and 'ID' in mask_df.columns:
        print(f"Merging {dataset_name} data...")
        joined = pd.merge(original_df, mask_df, on='ID', suffixes=('_img', '_mask'))
        joined = joined.rename(columns={
            'Location_img': 'ImagePath',
            'Location_mask': 'MaskPath', 
            'Class_img': 'ImageClass',
            'Class_mask': 'MaskClass',
            'Dataset_img': 'Dataset',
            'Type_img': 'Type'
        })
        # Keep only necessary columns
        joined = joined[['ID', 'ImagePath', 'MaskPath', 'SubClass', 'Dataset', 'MaskClass', 'Type']]
        
        # Clean subclass for IDRiD if needed
        if dataset_name == 'IDRiD':
            # Clean IDRiD subclass names
            joined['SubClassClean'] = joined['SubClass'].apply(
                lambda x: x.split('. ')[1].strip() if '. ' in str(x) else str(x)
            )
        else:
            # DDR already clean
            joined['SubClassClean'] = joined['SubClass']
        
        print(f"Joined {dataset_name} shape: {joined.shape}")
        return joined
    else:
        print(f"Skipping {dataset_name} merge - missing data or ID column")
        return pd.DataFrame()

print("\n=== Merging DataFrames ===")

# Merge all datasets
joined_idrid = merge_datasets(df_original_idrid, df_mask_idrid, 'IDRiD')
joined_ddr_train = merge_datasets(df_original_ddr_train, df_mask_ddr_train, 'DDR Train')
joined_ddr_val = merge_datasets(df_original_ddr_val, df_mask_ddr_val, 'DDR Validation')

# Combine all datasets
all_datasets = []
if len(joined_idrid) > 0:
    all_datasets.append(joined_idrid)
if len(joined_ddr_train) > 0:
    all_datasets.append(joined_ddr_train)
if len(joined_ddr_val) > 0:
    all_datasets.append(joined_ddr_val)

if all_datasets:
    combined_df = pd.concat(all_datasets, ignore_index=True)
    print(f"\nCombined dataset shape: {combined_df.shape}")
    
    # Show distribution by dataset and type
    print("\nDataset distribution:")
    print(combined_df.groupby(['Dataset', 'Type']).size())
    
    print("\nClass distribution:")
    print(combined_df['MaskClass'].value_counts())
else:
    combined_df = pd.DataFrame()
    print("No datasets were successfully merged")

print("\n=== Final DataFrames ===")
print(f"Combined dataset: {combined_df.shape if len(combined_df) > 0 else 'Empty'}")

# Save the datasets
if len(joined_idrid) > 0:
    joined_idrid.to_csv('joined_idrid.csv', index=False)
    print("Saved joined_idrid.csv")

if len(joined_ddr_train) > 0:
    joined_ddr_train.to_csv('joined_ddr_train.csv', index=False)
    print("Saved joined_ddr_train.csv")

if len(joined_ddr_val) > 0:
    joined_ddr_val.to_csv('joined_ddr_val.csv', index=False)
    print("Saved joined_ddr_val.csv")

if len(combined_df) > 0:
    combined_df.to_csv('combined_dataset.csv', index=False)
    print("Saved combined_dataset.csv")

# Now create train/val splits properly
def create_final_splits(combined_df, test_size=0.3, random_state=42):
    """
    Create proper train/validation splits considering the existing splits
    """
    # For IDRiD, we need to split since it doesn't have predefined validation
    idrid_data = combined_df[combined_df['Dataset'] == 'idrid']
    ddr_train = combined_df[(combined_df['Dataset'] == 'ddr') & (combined_df['Type'] == 'train')]
    ddr_val = combined_df[(combined_df['Dataset'] == 'ddr') & (combined_df['Type'] == 'validation')]
    
    print(f"\nCreating final splits:")
    print(f"IDRiD data: {len(idrid_data['ID'].unique())} images")
    print(f"DDR train: {len(ddr_train['ID'].unique())} images") 
    print(f"DDR validation: {len(ddr_val['ID'].unique())} images")
    
    # Split IDRiD into train/val
    idrid_ids = idrid_data['ID'].unique()
    idrid_train_ids, idrid_val_ids = train_test_split(
        idrid_ids, test_size=test_size, random_state=random_state, shuffle=True
    )
    
    # Create final DataFrames
    train_df = pd.concat([
        idrid_data[idrid_data['ID'].isin(idrid_train_ids)],
        ddr_train
    ], ignore_index=True)
    
    val_df = pd.concat([
        idrid_data[idrid_data['ID'].isin(idrid_val_ids)],
        ddr_val
    ], ignore_index=True)
    
    print(f"\nFinal splits:")
    print(f"Training: {len(train_df['ID'].unique())} images")
    print(f"Validation: {len(val_df['ID'].unique())} images")
    
    return train_df, val_df

# Create final splits
if len(combined_df) > 0:
    train_df, val_df = create_final_splits(combined_df)
    
    # Save the final splits
    train_df.to_csv('final_train_dataset.csv', index=False)
    val_df.to_csv('final_val_dataset.csv', index=False)
    print("\nSaved final_train_dataset.csv and final_val_dataset.csv")

In [ ]:
#Just for visualization
import cv2

# Define class colors (same as earlier)
CLASS_COLORS = {
    "MA": (0, 0, 255),    # Red
    "HE": (0, 255, 0),    # Green
    "EX": (255, 0, 0),    # Blue
    "SE": (255, 255, 0),  # Cyan
    "OD": (255, 0, 255)   # Magenta
}

CLASS_NAMES = ["MA", "HE", "EX", "SE", "OD"]

In [ ]:
def rgb_to_class(mask_rgb):
    """
    Convert an RGB mask image to integer mask with class IDs.
    
    Args:
        mask_rgb (np.array): HxWx3 RGB mask
    Returns:
        mask_class (np.array): HxW integer mask (0=background, 1=MA, 2=HE, etc.)
    """
    mask_class = np.zeros(mask_rgb.shape[:2], dtype=np.uint8)  # initialize background = 0
    for idx, class_name in enumerate(CLASS_NAMES, start=1):  # IDs start at 1
        color = CLASS_COLORS[class_name]
        matches = np.all(mask_rgb == color, axis=-1)
        mask_class[matches] = idx
    return mask_class

Defines a function named Create_Directory_Array, which is used to read metadata of TIFF image files from a specified directory structure (including class label, subclass name, full file path, and an ID extracted from the filename). These pieces of information are stored as a list of dictionaries. The list is then converted into a Pandas DataFrame, the rows of the DataFrame are randomly shuffled, and finally the DataFrame is saved as a CSV file.

Prints the shape of the saved dataset and displays the first 20 rows for inspection.

Performs a grouped count (aggregation) based on the class type.

In [ ]:
#Wow look the data is so cool amirite
print(train_df.shape)
train_df.head(20)

In [ ]:
#Now we got the REAL class names
CLASS_NAMES = ["Microaneurysms", "Haemorrhages", "Hard Exudates", "Soft Exudates"]
n_classes = len(CLASS_NAMES)

In [ ]:

train_df[train_df['SubClassClean'].isna()]

In [ ]:
#PRetty self explanatory if u dont get it just slam it into chatgpt

def load_mask(mask_path, subclass_name, size=(512,512)):

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, size, interpolation=cv2.INTER_NEAREST)
    mask_class = np.zeros(mask.shape, dtype=np.uint8)
    if np.max(mask) > 0:
        class_id = CLASS_NAMES.index(subclass_name) + 1  # 1-based class ID
        mask_class[mask > 0] = class_id
        
    mask_class = np.expand_dims(mask_class, axis=-1)
    return mask_class

In [ ]:
#Same as above

def load_img(image_path, size=(512,512)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    img = img.astype(np.float32) / 255.0
    return img

In [ ]:
#This one combines the masks so we can overlay onto the image(The masks are stored as seperate images)

def combine_masks(df_group, size=(512,512)):
    combined_mask = np.zeros((size[1], size[0], 1), dtype=np.uint8)
    for _, row in df_group.iterrows():
        if row['SubClassClean'] == "Optic Disc":
            continue
        single_mask = load_mask(row['MaskPath'], row['SubClassClean'], size=size)
        combined_mask = np.maximum(combined_mask, single_mask)
    return combined_mask

In [ ]:
#Load... the data?

def load_data(joined_df, size=(512,512)):
    images = []
    masks = []
    #Find all images by id
    grouped = joined_df.groupby('ID')
    for img_id, group in grouped:
        # Load image
        image_path = group['ImagePath'].iloc[0]  # all rows have same image path
        images.append(load_img(image_path, size=size))
        
        # Combine all masks for this image
        masks.append(combine_masks(group, size=size))
        
    return np.array(images, dtype=np.float32), np.array(masks, dtype=np.uint8)

In [ ]:
X, Y = load_data(train_df)
print("Images shape:", X.shape)
print("Masks shape:", Y.shape)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Define class colors and names
CLASS_INFO = {
    1: ("Microaneurysms", (255, 0, 0)),      # Red
    2: ("Haemorrhages", (0, 255, 0)),        # Green
    3: ("Hard Exudates", (0, 0, 255)),       # Blue
    4: ("Soft Exudates", (255, 255, 0)),     # Yellow 
}

#Wow so much work just to see a couple eyeball pics 
#I generated this whole function with chatgpt
def visualize_image_with_mask(image, mask):
    """
    image: numpy array (H,W,3), float32 [0,1] RGB
    mask: numpy array (H,W,1), uint8 class IDs (0 = background)
    """
    mask = mask.squeeze()
    overlay = np.zeros_like(image, dtype=np.uint8)

    # Build overlay per class
    for class_id, (_, color) in CLASS_INFO.items():
        overlay[mask == class_id] = color

    # Display side by side
    plt.figure(figsize=(12,6))

    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(image)
    plt.imshow(overlay, alpha=0.5)
    plt.title("Image with Mask Overlay")
    plt.axis('off')

    # Legend patches
    patches = [mpatches.Patch(color=np.array(color)/255.0, label=name) 
               for _, (name, color) in CLASS_INFO.items()]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.show()

In [ ]:
# Pick a sample from the loaded dataset
sample_idx = 10
visualize_image_with_mask(X[sample_idx], Y[sample_idx])

**Now we get to the good shit**

In [ ]:
#Import torch
import torch
import numpy as np

#We have to prepare our datasets for pytorch.
#Why not combine the other function with this one? -some nerd
def prepare_for_pytorch(X, Y):
    # X: (N, H, W, 3) -> (N, 3, H, W)
    X_torch = np.transpose(X, (0, 3, 1, 2)).astype(np.float32)
    
    # Y: (N, H, W, 1) -> (N, H, W), dtype long
    Y_torch = Y.squeeze(-1).astype(np.int64)
    
    # Convert to torch tensors
    X_torch = torch.from_numpy(X_torch)
    Y_torch = torch.from_numpy(Y_torch)
    
    return X_torch, Y_torch


X_torch, Y_torch = prepare_for_pytorch(X, Y)
print("Torch Images:", X_torch.shape, X_torch.dtype)  # (N, 3, H, W), float32
print("Torch Masks:", Y_torch.shape, Y_torch.dtype)   # (N, H, W), int64
print("Unique mask values:", torch.unique(Y_torch))

In [ ]:
#I think the kernel got turned off at some point so everytime i got a missing import error
#I added another import line. You can probably fix this up
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import numpy as np
import pandas as pd
import cv2
import os
from PIL import Image
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

#Important for image segmentation, tells us how close the AI mask is to the real one with no background
class DiceLoss(nn.Module):
    """
    Dice Loss for semantic segmentation tasks.
    
    Dice Loss (also known as F1 Score Loss or Sørensen–Dice Loss) measures the 
    overlap between predicted and ground truth segmentation masks. It's particularly 
    effective for imbalanced datasets where some classes have much fewer pixels.
    
    Formula for single class: Dice = (2 * |X ∩ Y|) / (|X| + |Y|)
    Dice Loss = 1 - Dice
    
    Properties:
    - Range: [0, 1] where 0 = perfect overlap, 1 = no overlap
    - Handles class imbalance better than CrossEntropy
    - Directly optimizes for intersection-over-union (IoU)
    """
    
    def __init__(self, smooth=1e-6):
        """
        Initialize Dice Loss.
        
        Args:
            smooth (float): Smoothing factor to avoid division by zero.
                           Prevents numerical instability when there are no
                           pixels of a particular class in either prediction or target.
        """
        super(DiceLoss, self).__init__()
        self.smooth = smooth  # Small epsilon for numerical stability

    def forward(self, predictions, targets):
        """
        Compute Dice Loss between predictions and targets.
        
        Args:
            predictions (torch.Tensor): Model outputs with shape (B, C, H, W)
                                        where C = number of classes.
                                        Typically comes from final conv layer without softmax.
            targets (torch.Tensor): Ground truth masks with shape (B, H, W)
                                    containing class indices (0 to C-1).
        
        Returns:
            torch.Tensor: Scalar Dice Loss value averaged over all non-background classes.
        """
        # Get number of classes from prediction tensor shape
        # predictions shape: (batch_size, n_classes, height, width)
        n_classes = predictions.shape[1]
        dice_loss = 0  # Accumulator for total dice loss
        
        # Convert targets from class indices to one-hot encoding
        # targets shape: (B, H, W) → (B, H, W, C) → (B, C, H, W)
        targets_one_hot = F.one_hot(targets, n_classes)  # (B, H, W, C)
        targets_one_hot = targets_one_hot.permute(0, 3, 1, 2).float()  # (B, C, H, W)
        
        # Calculate Dice loss for each class (excluding background class 0)
        for class_id in range(1, n_classes):  # Skip background (class 0)
            # Get predicted probabilities for current class
            # Shape: (B, H, W) - model's confidence for this class at each pixel
            pred = predictions[:, class_id]
            
            # Get ground truth binary mask for current class  
            # Shape: (B, H, W) - 1 where class exists, 0 elsewhere
            target = targets_one_hot[:, class_id]
            
            # Calculate intersection: sum of element-wise product
            # Measures overlap between prediction and ground truth
            intersection = (pred * target).sum()
            
            # Calculate union: sum of individual areas
            # Represents total area covered by both prediction and ground truth
            union = pred.sum() + target.sum()
            
            # Compute Dice coefficient with smoothing
            # Smoothing prevents division by zero when no pixels of this class exist
            dice = (2. * intersection + self.smooth) / (union + self.smooth)
            
            # Convert to loss: 1 - Dice coefficient
            # We want to maximize Dice, so we minimize (1 - Dice)
            dice_loss += 1 - dice
        
        # Return average Dice loss over all non-background classes
        # Normalize by (n_classes - 1) because we skipped background
        return dice_loss / (n_classes - 1)

class FocalLoss(nn.Module):
    """
    Focal Loss for addressing class imbalance by down-weighting easy examples
    and focusing training on hard misclassified examples.
    
    Based on: "Focal Loss for Dense Object Detection" (Lin et al., 2017)
    Formula: FL(p_t) = -α * (1 - p_t)^γ * log(p_t)
    """
    
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        """
        Args:
            alpha (float): Weighting factor for class balancing. 
                          α < 1 for majority class, α > 1 for minority class.
            gamma (float): Focusing parameter. 
                          γ > 0 reduces loss for well-classified examples.
            reduction (str): 'mean', 'sum', or 'none' for loss reduction.
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        Args:
            inputs (Tensor): Model predictions (logits), shape (B, C, H, W)
            targets (Tensor): Ground truth labels, shape (B, H, W)
        
        Returns:
            Tensor: Computed focal loss
        """
        # Standard cross-entropy loss (pixel-wise)
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        
        # p_t = probability of true class → higher for confident correct predictions
        pt = torch.exp(-ce_loss)
        
        # Focal loss: down-weight easy examples by (1-p_t)^gamma
        # - Well-classified (p_t → 1): loss reduced significantly
        # - Misclassified (p_t → 0): loss largely unchanged
        focal_loss = self.alpha * (1 - pt)**self.gamma * ce_loss
        
        # Apply reduction
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

#Combine them. Dice weight tells it how much to weigh the dice loss btw.
class CombinedLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, dice_weight=0.7):
        super(CombinedLoss, self).__init__()
        self.focal = FocalLoss(alpha=alpha, gamma=gamma)
        self.dice = DiceLoss()
        self.dice_weight = dice_weight

    def forward(self, inputs, targets):
        focal_loss = self.focal(inputs, targets)
        dice_loss = self.dice(inputs, targets)
        return (1 - self.dice_weight) * focal_loss + self.dice_weight * dice_loss

In [ ]:

#Just finds the class of the anomalies through their names
def infer_class_from_path(mask_path):
    path = mask_path.replace("\\", "/").lower()
    filename = os.path.basename(path)

    # 1. DDR folder-based detection (critical!)
    if '/ma/' in path.lower():
        return 1
    if '/he/' in path.lower():
        return 2
    if '/ex/' in path.lower():
        return 3
    if '/se/' in path.lower():
        return 4

    # 2. IDRiD folder-based
    if '/1. microaneurysms/' in path:
        return 1
    if '/2. haemorrhages/' in path:
        return 2
    if '/3. hard exudates/' in path:
        return 3
    if '/4. soft exudates/' in path:
        return 4
    if '/5. optic disc/' in path:
        return 5

    # 3. IDRiD filename-based (suffix patterns)
    if '_ma' in filename:
        return 1
    if '_he' in filename:
        return 2
    if '_ex' in filename:
        return 3
    if '_se' in filename:
        return 4
    if '_od' in filename:
        return 5

    print(f"Warning: Could not infer class from path: {mask_path}")
    return 1

def _get_class_name(class_id):
    """Get class name from class ID"""
    class_names = {
        1: "Microaneurysms",
        2: "Haemorrhages", 
        3: "Hard Exudates",
        4: "Soft Exudates",
        5: "Optic Disc"
    }
    return class_names.get(class_id, "Unknown")

def combine_masks_by_image(df, img_col='ImagePath', mask_col='MaskPath'):
    """Group DataFrame by image and combine all masks for each image"""
    print("Combining masks by image...")
    
    # Group by image and collect all mask paths
    grouped = df.groupby(img_col)[mask_col].apply(list).reset_index()
    
    combined_data = []
    for _, row in grouped.iterrows():
        img_path = row[img_col]
        mask_paths = row[mask_col]
        
        # Create a dictionary for this image with all its masks
        img_data = {'ImagePath': img_path}
        
        # Classify each mask path
        for mask_path in mask_paths:
            class_id = infer_class_from_path(mask_path)
            class_name = _get_class_name(class_id).replace(' ', '')
            img_data[f'{class_name}_path'] = mask_path
        
        combined_data.append(img_data)
    
    combined_df = pd.DataFrame(combined_data)
    print(f"Combined {len(df)} rows -> {len(combined_df)} images")
    return combined_df

#Combine masks for the data
class RetinopathyCombinedDatasetSimple(Dataset):
    def __init__(self, df, img_col='ImagePath', mask_dict=None, size=(512, 512), augment=False):
        self.df = df.copy()
        self.img_col = img_col
        self.mask_dict = mask_dict or {
            "Microaneurysms": "Microaneurysms_path",
            "Haemorrhages": "Haemorrhages_path", 
            "Hard Exudates": "HardExudates_path",
            "Soft Exudates": "SoftExudates_path",
            "Optic Disc": "OpticDisc_path"
        }
        self.size = size
        self.augment = augment

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load image
        image = cv2.imread(row[self.img_col])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Resize image to target size
        image = cv2.resize(image, (self.size[1], self.size[0]))
        
        # Create empty mask
        combined_mask = np.zeros((self.size[0], self.size[1]), dtype=np.uint8)
        
        # Load and add each class
        for class_id, (class_name, mask_col) in enumerate(self.mask_dict.items(), start=1):
            mask_path = row[mask_col]
            
            if pd.isna(mask_path) or mask_path == '':
                continue
                
            try:
                class_mask = np.array(Image.open(mask_path))
                if len(class_mask.shape) == 3:
                    class_mask = class_mask[:, :, 0]
                
                # Resize to target size
                class_mask = cv2.resize(class_mask, (self.size[1], self.size[0]), 
                                      interpolation=cv2.INTER_NEAREST)
                
                # Add to combined mask
                combined_mask[class_mask > 0] = class_id
                
            except Exception as e:
                print(f"Error loading {class_name}: {e}")
                continue
        
        # Convert to tensor
        image_tensor = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
        mask_tensor = torch.from_numpy(combined_mask).long()
        
        return image_tensor, mask_tensor

**MODEL ARCHITECTURE**

In [ ]:
class ConvBNReLU(nn.Module):
    def __init__(self, n_channels, out_ch, k=3, stride=1, padding=None, groups=1):
        super().__init__()
        if padding is None:
            padding = (k - 1) // 2
        self.conv = nn.Conv2d(n_channels, out_ch, k, stride=stride, padding=padding, groups=groups, bias=False)
        self.bn = nn.BatchNorm2d(out_ch)
        self.act = nn.GELU()
    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class DepthwiseSeparable(nn.Module):
    def __init__(self, n_channels, out_ch, k=3, stride=1):
        super().__init__()
        pad = (k-1)//2
        self.dw = nn.Conv2d(n_channels, n_channels, kernel_size=k, stride=stride, padding=pad, groups=n_channels, bias=False)
        self.pw = nn.Conv2d(n_channels, out_ch, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_ch)
        self.act = nn.GELU()
    def forward(self, x):
        x = self.dw(x)
        x = self.pw(x)
        x = self.bn(x)
        return self.act(x)

# -------------------------
# Polarized Self-Attention (PSA)
# -------------------------
class PolarizedSelfAttention(nn.Module):
    def __init__(self, channels, reduction=4):
        super().__init__()
        self.channels = channels
        
        self.channel_pool = nn.AdaptiveAvgPool2d(1)
        self.channel_fc = nn.Sequential(
            nn.Linear(channels, channels // reduction),
            nn.GELU(),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        )
        
        self.spatial_conv = nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False)
        self.spatial_sigmoid = nn.Sigmoid()
    
    def channel_attention(self, x):
        b, c, _, _ = x.size()
        y = self.channel_pool(x).view(b, c)
        y = self.channel_fc(y).view(b, c, 1, 1)
        return x * y
    
    def spatial_attention(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        y = self.spatial_conv(y)
        y = self.spatial_sigmoid(y)
        return x * y
    
    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x

# -------------------------
# Mobile Attention Convolution (MAC)
# -------------------------
class MAC(nn.Module):
    def __init__(self, n_channels, out_ch, expansion=4):
        super().__init__()
        hidden_dim = n_channels * expansion
        
        self.expand = nn.Conv2d(n_channels, hidden_dim, 1, bias=False) if n_channels != hidden_dim else nn.Identity()
        self.bn1 = nn.BatchNorm2d(hidden_dim)
        
        self.dw_conv = nn.Conv2d(hidden_dim, hidden_dim, 3, padding=1, groups=hidden_dim, bias=False)
        self.bn2 = nn.BatchNorm2d(hidden_dim)
        
        self.psa = PolarizedSelfAttention(hidden_dim)
        
        self.project = nn.Conv2d(hidden_dim, out_ch, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_ch)
        
        self.act = nn.GELU()
        self.use_residual = (n_channels == out_ch)
    
    def forward(self, x):
        identity = x
        
        x = self.expand(x)
        x = self.bn1(x)
        x = self.act(x)
        
        x = self.dw_conv(x)
        x = self.bn2(x)
        x = self.act(x)
        
        x = self.psa(x)
        
        x = self.project(x)
        x = self.bn3(x)
        
        if self.use_residual:
            x = x + identity
            
        return self.act(x)

# -------------------------
# Multi-scale Large-kernel Dual Attention (MLDA)
# -------------------------
class MLDA(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.channels = channels
        
        self.local_conv = nn.Conv2d(channels, channels, 1, bias=False)
        self.local_bn = nn.BatchNorm2d(channels)
        
        self.channel_att = PolarizedSelfAttention(channels)
        
        self.branch0 = DepthwiseSeparable(channels, channels, k=5)
        
        self.branch1 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=(7, 1), padding=(3, 0), groups=channels, bias=False),
            nn.Conv2d(channels, channels, kernel_size=(1, 7), padding=(0, 3), groups=channels, bias=False),
            nn.BatchNorm2d(channels),
            nn.GELU()
        )
        
        self.branch2 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=(11, 1), padding=(5, 0), groups=channels, bias=False),
            nn.Conv2d(channels, channels, kernel_size=(1, 11), padding=(0, 5), groups=channels, bias=False),
            nn.BatchNorm2d(channels),
            nn.GELU()
        )
        
        self.branch3 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=(21, 1), padding=(10, 0), groups=channels, bias=False),
            nn.Conv2d(channels, channels, kernel_size=(1, 21), padding=(0, 10), groups=channels, bias=False),
            nn.BatchNorm2d(channels),
            nn.GELU()
        )
        
        self.fuse = nn.Conv2d(channels * 4, channels, 1, bias=False)
        self.fuse_bn = nn.BatchNorm2d(channels)
        self.fuse_act = nn.GELU()
        
        self.final_conv = nn.Conv2d(channels, channels, 1, bias=False)
        
    def forward(self, x):
        local = self.local_conv(x)
        local = self.local_bn(local)
        
        x_att = self.channel_att(local)
        
        b0 = self.branch0(x_att)
        b1 = self.branch1(x_att)
        b2 = self.branch2(x_att)
        b3 = self.branch3(x_att)
        
        multi_scale = torch.cat([b0, b1, b2, b3], dim=1)
        fused = self.fuse(multi_scale)
        fused = self.fuse_bn(fused)
        fused = self.fuse_act(fused)
        
        out = self.final_conv(fused)
        return out

# -------------------------
# Mobile Multi-scale Attention Convolution (MMAC)
# -------------------------
class MMAC(nn.Module):
    def __init__(self, n_channels, out_ch):
        super().__init__()
        self.mac1 = MAC(n_channels, out_ch)
        self.mac2 = MAC(out_ch, out_ch)
        self.mlda = MLDA(out_ch)
        
    def forward(self, x):
        x = self.mac1(x)
        x = self.mac2(x)
        x = self.mlda(x)
        return x

# -------------------------
# CPCF fusion module - FIXED VERSION
# -------------------------
class ResizeOp(nn.Module):
    """Learnable resize operation with projection to target channels"""
    def __init__(self, n_channels, out_ch, mode='up'):
        super().__init__()
        self.mode = mode
        self.conv = nn.Conv2d(n_channels, out_ch, 1, bias=False)
        
    def forward(self, x):
        x = self.conv(x)
        if self.mode == 'up':
            return F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)
        else:
            return F.avg_pool2d(x, kernel_size=2, stride=2)

class CPCFModule(nn.Module):
    """
    Fixed CPCF module with proper channel projection
    """
    def __init__(self, target_channels, stage, f1_ch, f2_ch, f3_ch, f4_ch):
        super().__init__()
        self.stage = stage
        self.target_channels = target_channels
        
        # Projection layers for each input feature to target_channels
        self.proj1 = nn.Conv2d(f1_ch, target_channels, 1, bias=False)
        self.proj2 = nn.Conv2d(f2_ch, target_channels, 1, bias=False)
        self.proj3 = nn.Conv2d(f3_ch, target_channels, 1, bias=False)
        self.proj4 = nn.Conv2d(f4_ch, target_channels, 1, bias=False)
        
        # Resize operations
        self.rup = ResizeOp(target_channels, target_channels, mode='up')
        self.rdown = ResizeOp(target_channels, target_channels, mode='down')
        
        # Attention block
        self.att = MLDA(target_channels)
        
    def forward(self, f1, f2, f3, f4):
        # Project all features to target channel dimension
        f1_p = self.proj1(f1)
        f2_p = self.proj2(f2)
        f3_p = self.proj3(f3)
        f4_p = self.proj4(f4)
        
        # Implement the exact fusion strategy from paper Eq.(3)
        if self.stage == 1:
            # S1 = Rup(Rup(Rup(f4) + f3) + f2) + f1
            s = self.rup(self.rup(self.rup(f4_p) + f3_p) + f2_p) + f1_p
            ref_feature = f1_p
        elif self.stage == 2:
            # S2 = Rup(Rup(f4) + f3) + Rdown(f1) + f2
            s = self.rup(self.rup(f4_p) + f3_p) + self.rdown(f1_p) + f2_p
            ref_feature = f2_p
        elif self.stage == 3:
            # S3 = Rdown(Rdown(f1) + f2) + Rup(f4) + f3
            s = self.rdown(self.rdown(f1_p) + f2_p) + self.rup(f4_p) + f3_p
            ref_feature = f3_p
        elif self.stage == 4:
            # S4 = Rdown(Rdown(Rdown(f1) + f2) + f3) + f4
            s = self.rdown(self.rdown(self.rdown(f1_p) + f2_p) + f3_p) + f4_p
            ref_feature = f4_p
        
        # Apply attention and skip connection
        m = self.att(s) + ref_feature
        return m

# -------------------------
# Complete CMAC-Net
# -------------------------
class CMACNet(nn.Module):
    def __init__(self, n_channels=3, base_ch=32, n_classes=5):
        super().__init__()
        
        # -------------------------
        # Encoder (MDAC Backbone)
        # -------------------------
        self.stem = nn.Sequential(
            nn.Conv2d(n_channels, base_ch, 3, stride=2, padding=1),
            nn.Conv2d(base_ch, base_ch, 3, stride=2, padding=1),
            nn.BatchNorm2d(base_ch),
            nn.GELU()
        )
        
        # Stage 1
        self.stage1 = nn.Sequential(MAC(base_ch, base_ch))
        self.pool1 = nn.MaxPool2d(2)
        
        # Stage 2
        self.stage2 = nn.Sequential(
            MAC(base_ch, base_ch * 2),
            MAC(base_ch * 2, base_ch * 2)
        )
        self.pool2 = nn.MaxPool2d(2)
        
        # Stage 3
        self.stage3 = nn.Sequential(
            MMAC(base_ch * 2, base_ch * 4),
            MMAC(base_ch * 4, base_ch * 4),
            MMAC(base_ch * 4, base_ch * 4)
        )
        self.pool3 = nn.MaxPool2d(2)
        
        # Stage 4
        self.stage4 = nn.Sequential(
            MMAC(base_ch * 4, base_ch * 8),
            MMAC(base_ch * 8, base_ch * 8),
            MMAC(base_ch * 8, base_ch * 8),
            MMAC(base_ch * 8, base_ch * 8),
            MMAC(base_ch * 8, base_ch * 8),
            MMAC(base_ch * 8, base_ch * 8)
        )
        self.pool4 = nn.MaxPool2d(2)
        
        # Stage 5 (bottleneck)
        self.stage5 = nn.Sequential(
            MMAC(base_ch * 8, base_ch * 16),
            MMAC(base_ch * 16, base_ch * 16),
            MMAC(base_ch * 16, base_ch * 16)
        )
        
        # -------------------------
        # CPCF Modules with proper channel handling
        # -------------------------
        # Define the actual channel dimensions for each stage
        f1_ch = base_ch      # Stage 1 output channels
        f2_ch = base_ch * 2  # Stage 2 output channels  
        f3_ch = base_ch * 4  # Stage 3 output channels
        f4_ch = base_ch * 8  # Stage 4 output channels
        
        self.cpcf1 = CPCFModule(f1_ch, stage=1, f1_ch=f1_ch, f2_ch=f2_ch, f3_ch=f3_ch, f4_ch=f4_ch)
        self.cpcf2 = CPCFModule(f2_ch, stage=2, f1_ch=f1_ch, f2_ch=f2_ch, f3_ch=f3_ch, f4_ch=f4_ch)
        self.cpcf3 = CPCFModule(f3_ch, stage=3, f1_ch=f1_ch, f2_ch=f2_ch, f3_ch=f3_ch, f4_ch=f4_ch)
        self.cpcf4 = CPCFModule(f4_ch, stage=4, f1_ch=f1_ch, f2_ch=f2_ch, f3_ch=f3_ch, f4_ch=f4_ch)
        
        # -------------------------
        # Decoder
        # -------------------------
        self.up5 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            ConvBNReLU(base_ch * 16, base_ch * 8, k=3) 
        )
        
        self.up4 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            ConvBNReLU(base_ch * 8, base_ch * 4, k=3) 
        )
        
        self.up3 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            ConvBNReLU(base_ch * 4, base_ch * 2, k=3)  
        )
        
        self.up2 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            ConvBNReLU(base_ch * 2, base_ch, k=3)  
        )
        
        self.up1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            ConvBNReLU(base_ch, base_ch, k=3)
        )
        
        # Final output
        self.final_upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.final_conv = nn.Conv2d(base_ch, n_classes, 1)
        
    def forward(self, x):
        # Initial stem
        x = self.stem(x)
        
        # Encoder forward pass
        f1 = self.stage1(x)           # base_ch
        f2 = self.stage2(self.pool1(f1))  # base_ch * 2
        f3 = self.stage3(self.pool2(f2))  # base_ch * 4
        f4 = self.stage4(self.pool3(f3))  # base_ch * 8
        f5 = self.stage5(self.pool4(f4))  # base_ch * 16
        
        # CPCF fusion
        m1 = self.cpcf1(f1, f2, f3, f4)
        m2 = self.cpcf2(f1, f2, f3, f4)
        m3 = self.cpcf3(f1, f2, f3, f4)
        m4 = self.cpcf4(f1, f2, f3, f4)
        
        # Decoder with skip connections
        d4 = self.up5(f5) + m4
        d3 = self.up4(d4) + m3
        d2 = self.up3(d3) + m2
        d1 = self.up2(d2) + m1
        d0 = self.up1(d1)
        
        # Final output
        out = self.final_upsample(d0)
        out = self.final_conv(out)
        return out

In [ ]:
# Assuming your cmac model is defined as `model`
n_classes = len(CLASS_NAMES) + 1  # background + subclasses
model = CMACNet(n_channels=3, n_classes=n_classes)  # initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def calculate_iou_per_class(predictions, targets, n_classes=6):
    """Calculate IoU for each class separately"""
    ious = []
    predictions = torch.argmax(predictions, dim=1)
    
    for class_id in range(1, n_classes):  # Skip background
        pred_mask = (predictions == class_id)
        target_mask = (targets == class_id)
        
        intersection = (pred_mask & target_mask).float().sum()
        union = (pred_mask | target_mask).float().sum()
        
        if union > 0:
            iou = intersection / union
        else:
            iou = torch.tensor(0.0)
        ious.append(iou.item())
    
    return ious

def create_weighted_sampler(dataset):
    """Create sampler that oversamples images with rare pathologies"""
    weights = []
    
    for i, (_, mask) in enumerate(dataset):
        unique_classes = torch.unique(mask)
        
        # Higher weight if image contains rare pathologies (classes 1-4)
        rare_classes_present = any(cls in unique_classes for cls in [1, 2, 3, 4])
        weight = 5.0 if rare_classes_present else 1.0
        weights.append(weight)
    
    sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
    return sampler

def train_one_epoch_with_gradient_control(model, dataloader, optimizer, criterion, device, n_classes=6, max_grad_norm=1.0):
    model.train()
    running_loss = 0.0
    class_ious = [[] for _ in range(n_classes-1)]
    gradient_norms = []
    
    loop = tqdm(dataloader, leave=False)
    for imgs, masks in loop:
        imgs, masks = imgs.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, masks)
        loss.backward()
        
        # Gradient clipping
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        gradient_norms.append(grad_norm.item())
        
        optimizer.step()

        # Calculate metrics
        batch_ious = calculate_iou_per_class(outputs, masks, n_classes)
        for i, iou in enumerate(batch_ious):
            class_ious[i].append(iou)

        running_loss += loss.item()
        loop.set_description(f"Training (Grad: {grad_norm:.2f})")
        loop.set_postfix(loss=loss.item())
    
    # Print metrics
    avg_ious = [np.mean(iou_list) for iou_list in class_ious]
    class_names = ["MA", "HE", "EX", "SE", "OD"]
    print("Per-class IoU:", {name: f"{iou:.3f}" for name, iou in zip(class_names, avg_ious)})
    print(f"Avg Gradient Norm: {np.mean(gradient_norms):.3f}")
    
    return running_loss / len(dataloader)

#Evaluate model performance
def evaluate(model, dataloader, criterion, device):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, masks in dataloader:
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, masks)
            val_loss += loss.item()
    return val_loss / len(dataloader)

import matplotlib.pyplot as plt
import numpy as np

# Enhanced training function with visualization
def train_model_enhanced(model, train_loader, val_loader, epochs, device, class_names, checkpoint_path="cmac_best_balanced.pth"):
    best_val_loss = float("inf")
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)  # Higher LR, AdamW

# Warmup scheduler
def get_warmup_scheduler(optimizer, warmup_epochs=10):
    def lr_lambda(epoch):
        if epoch < warmup_epochs:
            return float(epoch) / float(warmup_epochs)
        else:
            return 0.1  # Then use ReduceLROnPlateau
            return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

        warmup_scheduler = get_warmup_scheduler(optimizer, warmup_epochs=10)
        plateau_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=8, factor=0.5, min_lr=1e-7)
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("=" * 50)
        
        # Train with metrics
        train_loss = train_one_epoch_with_metrics(model, train_loader, optimizer, criterion, device)
        val_loss = evaluate(model, val_loader, criterion, device)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.2e}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), checkpoint_path)
            print(f"✅ Saved best model | Val Loss: {val_loss:.4f}")

        # Visualize predictions after each epoch
        print(f"\n📊 Visualization for Epoch {epoch+1}:")
        visualize_predictions(model, val_loader, class_names, num_samples=3)

# Also add a function to show class color mapping
def show_class_legend():
    """Show color mapping for each class"""
    class_info = {
        0: ("Background", "black"),
        1: ("Microaneurysms", "red"),
        2: ("Haemorrhages", "green"), 
        3: ("Hard Exudates", "blue"),
        4: ("Soft Exudates", "yellow"),
        5: ("Optic Disc", "magenta")
    }
    
    plt.figure(figsize=(8, 2))
    for i in range(6):
        plt.subplot(1, 6, i+1)
        color = plt.cm.tab10(i / 10)  # Using tab10 colormap
        plt.imshow([[i]], vmin=0, vmax=5, cmap="tab10")
        plt.title(f"{i}: {class_info[i][0]}", fontsize=8)
        plt.axis('off')
    plt.suptitle("Class Color Mapping", fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
# 1. Combine your data
combined_df = combine_masks_by_image(train_df)
print("\nCombined DataFrame columns:", combined_df.columns.tolist())

# 2. Create datasets
train_dataset = RetinopathyCombinedDatasetSimple(
    combined_df, 
    img_col='ImagePath',
    mask_dict={
        "Microaneurysms": "Microaneurysms_path",
        "Haemorrhages": "Haemorrhages_path",
        "Hard Exudates": "HardExudates_path", 
        "Soft Exudates": "SoftExudates_path",
        "Optic Disc": "OpticDisc_path"
    },
    size=(512, 512),
    augment=False
)

val_dataset = RetinopathyCombinedDatasetSimple(
    combined_df, 
    img_col='ImagePath',
    mask_dict={
        "Microaneurysms": "Microaneurysms_path",
        "Haemorrhages": "Haemorrhages_path", 
        "Hard Exudates": "HardExudates_path",
        "Soft Exudates": "SoftExudates_path",
        "Optic Disc": "OpticDisc_path"
    },
    size=(512, 512),
    augment=False
)

# 3. Create weighted sampler for training
train_sampler = create_weighted_sampler(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=8, sampler=train_sampler, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, pin_memory=True)

# 4. Setup model and training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_classes = 6  # background + 5 classes
model = CMACNet(n_channels=3, n_classes=n_classes).to(device)

# 5. Use balanced loss function
criterion = CombinedLoss(alpha=1, gamma=2, dice_weight=0.7)
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)

print(f"Training on: {device}")
print(f"Model configured for {n_classes} classes")
print("Using Combined Loss (Focal + Dice) for better class balance")

In [ ]:
# Update your main execution section:
# Define class names for visualization
CLASS_NAMES = ["Background", "Microaneurysms", "Haemorrhages", "Hard Exudates", "Soft Exudates", "Optic Disc"]

# Show class legend before training
print("Class Color Mapping:")
show_class_legend()

In [ ]:
for imgs, masks in train_loader:
    print("Batch images:", imgs.shape)  # (B, 3, H, W)
    print("Batch masks:", masks.shape)  # (B, H, W)
    print("Unique mask labels:", torch.unique(masks))
    break

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#Just a visualizer to see how the model performs after epoch
def visualize_predictions(model, dataloader, class_names, num_samples=4):
    model.eval()
    imgs, masks = next(iter(dataloader))  # take one batch
    imgs, masks = imgs.to(device), masks.to(device)

    with torch.no_grad():
        outputs = model(imgs)  # (B, C, H, W)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()  # take class with max logit

    imgs = imgs.cpu().numpy().transpose(0, 2, 3, 1)  # back to (B, H, W, C)
    masks = masks.cpu().numpy()

    plt.figure(figsize=(12, num_samples * 4))
    for i in range(num_samples):
        # Original image
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(imgs[i])
        plt.title("Image")
        plt.axis("off")

        # Ground truth
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(masks[i], cmap="tab20")
        plt.title("Ground Truth")
        plt.axis("off")

        # Prediction
        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(preds[i], cmap="tab20")
        plt.title("Prediction")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
def debug_predictions(model, dataloader, device):
    """Debug what the model is actually predicting"""
    model.eval()
    imgs, masks = next(iter(dataloader))
    imgs, masks = imgs.to(device), masks.to(device)
    
    with torch.no_grad():
        outputs = model(imgs)
        preds = torch.argmax(outputs, dim=1)
        probabilities = F.softmax(outputs, dim=1)
    
    print("\n🔍 PREDICTION DEBUG:")
    print(f"Output range: [{outputs.min().item():.3f}, {outputs.max().item():.3f}]")
    
    for class_id in range(6):
        class_probs = probabilities[:, class_id]
        class_preds = (preds == class_id).float()
        print(f"Class {class_id}: Avg prob {class_probs.mean().item():.3f}, Pred % {class_preds.mean().item():.3f}")
    
    # Check if any class gets reasonable predictions
    if (preds == 0).float().mean() > 0.95:
        print("🚨 WARNING: Model predicting mostly background!")
    elif preds.float().std() < 0.01:
        print("🚨 WARNING: Model predictions have very low variance!")

# Add this to your training loop
debug_predictions(model, val_loader, device)

In [ ]:
def train_model_fixed(model, train_loader, val_loader, epochs, device, class_names):
    # Start simple
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
    
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        print(f"\n🎯 Epoch {epoch+1}/{epochs}")
        print("=" * 60)
        
        # Debug before training
        if epoch == 0:
            print("Initial model state:")
            debug_predictions(model, val_loader, device)
        
        # Train
        train_loss = train_one_epoch_with_gradient_control(
            model, train_loader, optimizer, criterion, device, 
            max_grad_norm=1.0
        )
        
        # Validate
        val_loss = evaluate(model, val_loader, criterion, device)
        scheduler.step(val_loss)
        
        print(f"📊 Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        print(f"📈 Learning Rate: {optimizer.param_groups[0]['lr']:.2e}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "cmac_best.pth")
            print(f"✅ Saved best model | Val Loss: {val_loss:.4f}")
        
        # Debug predictions every few epochs
        if epoch % 5 == 0:
            debug_predictions(model, val_loader, device)
        
        # Visualize
        if epoch % 2 == 0:
            print(f"\n👀 Visualization Epoch {epoch+1}:")
            visualize_predictions(model, val_loader, class_names, num_samples=2)
        
        # If no progress after 10 epochs, try different approach
        if epoch > 10 and best_val_loss > 2.0:  # If loss is still very high
            print("🔄 No significant progress, trying weighted loss...")
            real_weights = calculate_class_weights_from_data(train_dataset)
            criterion = nn.CrossEntropyLoss(weight=real_weights.to(device))

# Reset model and start fresh
def reset_model():
    model = CMACNet(n_channels=3, n_classes=6).to(device)
    
    # Initialize weights properly
    def init_weights(m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
    
    model.apply(init_weights)
    return model

# Start fresh
#print("🔄 Starting fresh with proper initialization...")
#model = reset_model()
train_model_fixed(model, train_loader, val_loader, epochs=200, device=device, class_names=CLASS_NAMES)

In [ ]:
# Save
torch.save(model.state_dict(), "CMAC_weights.pth")